In [16]:
from os.path import join, splitext, basename
from os import listdir
import torch
from torch.optim.swa_utils import AveragedModel, update_bn
import numpy as np
import librosa
import torch.nn.functional as F
from tqdm import tqdm
from scipy.stats import mode
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
from natsort import natsorted
from sklearn.metrics import confusion_matrix, roc_auc_score
from coreml.utils.io import read_yml
from coreml.config import Config
from coreml.data.dataloader import get_dataloader
from coreml.models import factory as model_factory

In [2]:
config_name = 'competitions/2020/melanoma-classification/configs/effb5/best-1cycle-wd4e-1-384'

In [3]:
whole_train_2020 = pd.read_csv('/data/siim-isic-melanoma/raw/2020/train.csv')
whole_train_2019 = pd.read_csv('/data/siim-isic-melanoma/raw/2019/train.csv')
test_2020 = pd.read_csv('/data/siim-isic-melanoma/raw/2020/test.csv')

In [4]:
train = whole_train_2019[['image_name', 'target']].append(whole_train_2020[['image_name', 'target']])

In [5]:
train.head()

,image_name,target
0,ISIC_0000000,0
1,ISIC_0000001,0
2,ISIC_0000002,1
3,ISIC_0000003,0
4,ISIC_0000004,1


In [6]:
data_config_path = f'/data/siim-isic-melanoma/processed/versions/v3.0.0.yml'
print(f'Reading data config: {data_config_path}')
data_config = read_yml(data_config_path)

Reading data config: /data/siim-isic-melanoma/processed/versions/v3.0.0.yml


In [7]:
prediction_val = pd.read_csv(join('/output', config_name, 'logs/evaluation/val.csv'))

In [8]:
val = pd.DataFrame(data_config['val'])
val['image_name'] = val['file'].apply(lambda x: splitext(basename(x))[0])
val['label'] = val['label'].apply(lambda x: x['classification'])
val = val.drop(columns=['file'])

print('Shapes:')
print(len(prediction_val), len(val))
print()
    
val = pd.merge(prediction_val, val)
    
print('Performance without using SWA')
val_preds = val['target'].values
val_labels = val['label'].values
roc = roc_auc_score(val_labels, val_preds)
print(roc)

Shapes:
6536 6536

Performance without using SWA
0.9181316467934257


In [11]:
config = Config(join('/workspace/coreml', config_name + '.yml'))

In [12]:
val_dataloader, _ = get_dataloader(
        config.data, 'val',
        config.model['batch_size'],
        num_workers=10,
        shuffle=False,
        drop_last=False)

=> Loading dataset version file: [siim-isic-melanoma, v3.0.0, val]


Loading items: 100%|██████████| 6536/6536 [00:00<00:00, 454913.06it/s]


In [13]:
# set epoch
config.model['load']['version'] = config_name
config.model['load']['load_best'] = True

In [17]:
model = model_factory.create(config.model['name'], **{'config': config})

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b5_ra-9a3e5369.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b5_ra-9a3e5369.pth


SystemExit: [31mCheckpoint file does not exist at /output/competitions/2020/melanoma-classification/configs/effb5/best-1cycle-wd4e-1-384/checkpoints/best_ckpt.pth.tar[0m

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
